In [ ]:
reset -f


In [ ]:
import os
import pandas as pd
import numpy as np
import json
from datetime import datetime
from Utils import print_dst


In [ ]:
input_folder: str = 'InputData'
preprocessed_input_folder: str = 'PreprocessedData'
spot_price_file_name_gzip: str = 'Spot_Price_Table.gzip'
electricity_prices_file_name: str = "Electricity_Prices.json"
electricity_prices_file_name_gzip: str = 'Electricity_Price_Table.gzip'
electricity_prices_file_name_csv: str = 'Electricity_Price_Table.csv'


In [ ]:
file_path: str = os.path.join(preprocessed_input_folder, spot_price_file_name_gzip)
df_spot_price: pd.DataFrame = pd.read_parquet(file_path)
file_path: str = os.path.join(input_folder, electricity_prices_file_name)
file = open(file_path, "r")
df_electricity_prices: pd.DataFrame = pd.read_json(file, lines=True)
file.close()


In [ ]:
# df_electricity_prices.sort_values(["start_date"]).head(10)


In [ ]:
df_electricity_prices.dtypes


In [ ]:
df_spot_price.head()


In [ ]:
df_electricity_prices["start_date"] = pd.to_datetime(df_electricity_prices["start_date"])
df_electricity_prices["end_date"] = pd.to_datetime(df_electricity_prices["end_date"])


In [ ]:
columns: list = ["Siirto", "Unit", "start_date", "end_date", "alv"]
df_siirto: pd.DataFrame = df_electricity_prices[df_electricity_prices["Siirto"].notna()][columns]
columns: list = ["Unit", "start_date", "end_date", "alv", "Marginaali"]
df_marginaali: pd.DataFrame = df_electricity_prices[df_electricity_prices["Marginaali"].notna()][columns]
columns: list = ["Unit", "start_date", "end_date", "alv", "Energia"]
df_energia: pd.DataFrame = df_electricity_prices[df_electricity_prices["Energia"].notna()][columns]
columns: list = ["Unit", "start_date", "end_date", "alv", "Vero"]
df_vero: pd.DataFrame = df_electricity_prices[df_electricity_prices["Vero"].notna()][columns]


In [ ]:
df_siirto['start_date'] = df_siirto["start_date"].map(lambda start_date: print_dst(start_date))
df_siirto['end_date'] = df_siirto["end_date"].map(lambda end_date: print_dst(end_date))
df_siirto


In [ ]:
df_marginaali['start_date'] = df_marginaali["start_date"].map(lambda start_date: print_dst(start_date))
df_marginaali['end_date'] = df_marginaali["end_date"].map(lambda end_date: print_dst(end_date))
df_marginaali


In [ ]:
df_energia['start_date'] = df_energia["start_date"].map(lambda start_date: print_dst(start_date))
df_energia['end_date'] = df_energia["end_date"].map(lambda end_date: print_dst(end_date))
df_energia


In [ ]:
df_vero['start_date'] = df_vero["start_date"].map(lambda start_date: print_dst(start_date))
df_vero['end_date'] = df_vero["end_date"].map(lambda end_date: print_dst(end_date))
df_vero


In [ ]:
def initialize_electricity_hourly_price():
    start_date: str = "2022-01-01T00:00"
    end_date: str = datetime.now().strftime("%Y-%m-%dT23:00")
    start_date = print_dst(start_date)
    end_date = print_dst(end_date)
    date_hourly_range: list = pd.date_range(start_date, end_date, freq="H")
    df: pd.DataFrame = pd.DataFrame(date_hourly_range, columns=["DateTime"])
    print(df.head())
    return df


In [ ]:
def set_transmission_price(row):
    df: pd.DataFrame = df_siirto.loc[((df_siirto["end_date"] + \
            np.timedelta64(1,'D') > row["DateTime"]) | \
            (df_siirto["end_date"].isna())) & \
            ((df_siirto["start_date"] <= row["DateTime"]) | \
            (df_siirto["start_date"].isna()))]
    if df.shape[0] > 0:
        transmission_price = df.iloc[0]["Siirto"]
    else:
        print(row["DateTime"])
    return transmission_price


In [ ]:
def add_transmission_price(df):
    df["Siirto"] = df.apply(lambda row: set_transmission_price(row), axis=1)


In [ ]:
def set_marginal_price(row):
    df: pd.DataFrame = df_marginaali.loc[((df_marginaali["end_date"] + \
            np.timedelta64(1,'D') > row["DateTime"]) | \
            (df_marginaali["end_date"].isna())) & \
            ((df_marginaali["start_date"] <= row["DateTime"]) | \
            (df_marginaali["start_date"].isna()))]
    if df.shape[0] > 0:
        marginal_price = df.iloc[0]["Marginaali"]
    else:
        print(row["DateTime"])
    return marginal_price


In [ ]:
def add_marginal_price(df):
    df["Marginaali"] = df.apply(lambda row: set_marginal_price(row), axis=1)


In [ ]:
def set_energy_price(row):
    df: pd.DataFrame = df_energia.loc[((df_energia["end_date"] + \
            np.timedelta64(1,'D') > row["DateTime"]) | \
            (df_energia["end_date"].isna())) & \
            ((df_energia["start_date"] <= row["DateTime"]) | \
            (df_energia["start_date"].isna()))]
    if df.shape[0] > 0:
        if df.iloc[0]["Energia"] == "spot":
            df_energy_price: pd.Series = \
                df_spot_price[df_spot_price["DateTime"] == row["DateTime"]]
            if df_energy_price.shape[0] > 0:
                energy_price: float = df_energy_price.iloc[0]["Price 0%"]
            else:
                energy_price: None = None
        else:
            energy_price: float = df.iloc[0]["Energia"]
    else:
        print(row["DateTime"])
    return energy_price


In [ ]:
def add_energy_price(df):
    df["Energia"] = df.apply(lambda row: set_energy_price(row), axis=1)


In [ ]:
def set_tax_price(row):
    df: pd.DataFrame = df_vero.loc[
            ((df_vero["start_date"] <= row["DateTime"]) | \
            (df_vero["start_date"].isna()) & \
            ((df_vero["end_date"] + \
            np.timedelta64(1,'D') > row["DateTime"]) | \
            (df_vero["end_date"].isna())))]
    if df.shape[0] > 0:
        tax_price = df.iloc[0]["Vero"]
    else:
        print(row["DateTime"])
    return tax_price


In [ ]:
def add_tax_price(df):
    df["Vero"] = df.apply(lambda row: set_tax_price(row), axis=1)


In [ ]:
df_electricity_hourly_price: pd.DataFrame = initialize_electricity_hourly_price()


In [ ]:
add_energy_price(df_electricity_hourly_price)


In [ ]:
add_transmission_price(df_electricity_hourly_price)


In [ ]:
add_marginal_price(df_electricity_hourly_price)


In [ ]:
add_tax_price(df_electricity_hourly_price)


In [ ]:
def add_special_alv(energia, siirto, marginaali, vero):
    ALV1: int = 24
    ALV2: int = 10
    Summa_ALV = (siirto + marginaali + vero) * (1 + ALV1 / 100)
    Summa_ALV += (energia) * (1 + ALV2 / 100)
    return Summa_ALV


In [ ]:
def add_alv(energia, siirto, marginaali, vero):
    ALV: int = 24
    Summa_ALV = (siirto + marginaali + vero) * (1 + ALV / 100)
    Summa_ALV += (energia) * (1 + ALV / 100)
    return Summa_ALV


In [ ]:
def add_total_price_with_alv_24(df):
    start_date: str = '2022-12-01'
    end_date: str = '2023-04-30'
    df["ALV%"] = np.where((df['DateTime'].dt.strftime('%Y-%m-%d') >= start_date) & \
                              (df['DateTime'].dt.strftime('%Y-%m-%d') <= end_date) , "24/10", "24")
    df["Summa+ALV"] = np.where(df["ALV%"] == "24",
                               add_alv(df["Energia"], df["Siirto"], df["Marginaali"], df["Vero"]),
                               add_special_alv(df["Energia"], df["Siirto"], df["Marginaali"], df["Vero"]))


In [ ]:
df_electricity_hourly_price.head()


In [ ]:
add_total_price_with_alv_24(df_electricity_hourly_price)


In [ ]:
df_electricity_hourly_price.head()


In [ ]:
def save_preprocessed_electricity_price_df(df: pd.DataFrame):
    file_path: str = os.path.join(preprocessed_input_folder, electricity_prices_file_name_gzip)
    df.to_parquet(file_path, compression='gzip')
    file_path: str = os.path.join(preprocessed_input_folder, electricity_prices_file_name_csv)
    df.to_csv(file_path)


In [ ]:
save_preprocessed_electricity_price_df(df_electricity_hourly_price)


In [ ]:
pd.options.display.float_format = '{:,.2f}'.format
df_electricity_hourly_price.tail(200)


In [ ]:
df_electricity_hourly_price.shape


In [ ]:
df_electricity_hourly_price.iloc[6540:6580]


In [ ]:
df_electricity_hourly_price.iloc[7200:7260]


In [ ]:
df_electricity_hourly_price.iloc[8750:8775]
